In this jupyter notebook we explore DP CTGAN implementation on incident data.

# DP CTGAN

In [1]:
import sys
print(sys.version)

3.11.0 (main, Nov 17 2022, 11:48:18) [GCC 8.5.0 20210514 (Red Hat 8.5.0-10)]


## Setup

In [3]:
pip install --no-user dp_cgans

  Using cached dp_cgans-0.0.6-py3-none-any.whl.metadata (15 kB)
  Using cached copulas-0.12.3-py3-none-any.whl.metadata (9.5 kB)
  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached rdt-1.17.1-py3-none-any.whl.metadata (10 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached wandb-0.21.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached ctgan-0.11.0-py3-none-any.whl.metadata (10 kB)
  Using cached deepecho-0.7.0-py3-none-any.whl.metadata (10 kB)
  Using cached sdmetrics-0.22.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB

In [6]:
pip install --no-user matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 43.0 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 40.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade --no-user setuptools

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --no-user sdv==1.6.0

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --no-user rdt==1.9.0

  Using cached rdt-1.9.0-py2.py3-none-any.whl.metadata (63 kB)
Using cached rdt-1.9.0-py2.py3-none-any.whl (74 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import dp_cgans
import rdt
import torch

print("dp-cgans version:", dp_cgans.__version__)
print("rdt version:", rdt.__version__)
print("torch version:", torch.__version__)

/dcs/large/u2164966/MSc_Dissertation/myenv311/lib/python3.11/site-packages/copulas/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import iter_entry_points


dp-cgans version: 0.0.6
rdt version: 1.9.0
torch version: 2.8.0+cu128


## CODE

In [1]:
import pandas as pd

pd.options.display.max_rows = 4000

# Load your filtered dataset from pickle
DATA_PATH = "filtered_data/df_v2_filtered.pkl"

df = pd.read_pickle(DATA_PATH)

In [ ]:
import pandas as pd

cutoff = 5000

# Identify categories below the cutoff
counts = df['service_code_description'].value_counts()
low_categories = counts[counts < cutoff].index

# Replace low-frequency categories with "OTHER"
df['service_code_description'] = df['service_code_description'].replace(low_categories, 'OTHER')

# check new distribution
print(df['service_code_description'].value_counts().sort_values(ascending=False))

service_code_description
ASSIST MEMBER OF THE PUBLIC    169406
CONCERN FOR PERSON             121876
DISTURBANCE                     90395
THEFT                           87517
DOMESTIC INCIDENT               79291
ROAD TRAFFIC MATTER             75033
PUBLIC NUISANCE                 59037
ROAD TRAFFIC COLLISION          58259
OTHER                           44717
ABANDONED/SILENT 999 CALL       41102
EXTERNAL AGENCY REQUEST         40557
ASSAULT                         39608
NOISE                           36849
DAMAGE                          33240
NEIGHBOUR DISPUTE               30507
FALSE CALL                      30413
COMMUNICATIONS                  29706
MISSING PERSON/ABSCONDER        27140
SUSPECT PERSONS                 25141
POLICE INFORMATION              20941
FRAUD INCIDENT                  20360
ROAD TRAFFIC OFFENCE            18563
CHILD PROTECTION                17091
DRUGS/SUBSTANCE MISUSE          16948
VEHICLE CRIME                   15139
DOMESTIC BAIL CHECK      

In [ ]:
from pathlib import Path
import pandas as pd
import torch
from dp_cgans import DP_CGAN

def sanitize_filename(name: str) -> str:
    """Replace illegal filename characters with underscores."""
    return name.replace("/", "*").replace("\\", "*")
                                         
def train_dp_ctgan(df_slice: pd.DataFrame, service_code: str, epochs: int, outdir: Path) -> Path:
    """Train a DP-CTGAN model on one slice of data and save it."""
    outdir.mkdir(parents=True, exist_ok=True)
    model = DP_CGAN(
        epochs=epochs,
        batch_size=1024,
        generator_dim=(128, 128, 128),
        discriminator_dim=(128, 128, 128),
        generator_lr=2e-4,
        discriminator_lr=2e-4,
        discriminator_steps=1,
        verbose=True,
        private=True,   # Enable Differential Privacy
    )
    print(f"Training DP-CGAN for slice: {service_code}")
    model.fit(df_slice)
    safe_name = sanitize_filename(service_code)
    model_path = outdir / f"{safe_name}_dp_ctgan.pkl"
    model.save(str(model_path))
    
    # GPU memory cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return model_path

if __name__ == "__main__":
    CATEGORY_COL = "service_code_description"
    EPOCHS = 200
    OUTDIR = Path("DPCTGAN/models")
    
    # Check GPU availability
    if torch.cuda.is_available():
        torch.cuda.set_device(0)
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠ No GPU detected — training will run on CPU.")
    categories = df[CATEGORY_COL].value_counts().index.tolist()
    print(f"Launching DP-CTGAN training for {len(categories)} slices sequentially...")
    for code in categories:
        slice_df = df[df[CATEGORY_COL] == code].copy()
        model_path = train_dp_ctgan(slice_df, code, EPOCHS, OUTDIR)
        print(f"✅ Saved model: {model_path}")

Using GPU: NVIDIA GeForce RTX 2080 Ti
Launching DP-CTGAN training for 36 slices sequentially...
Training DP-CGAN for slice: ASSIST MEMBER OF THE PUBLIC


/dcs/large/u2164966/MSc_Dissertation/myenv311/lib/python3.11/site-packages/sklearn/mixture/_base.py:275: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
